# Create Slice for Fabric Rack
This is the slice creation for the ```Ryu & Mininet``` tutorial. This is the first step to create the slice on fabric using fablib.

<b> Prerequisites  
    
* You need to have your FABRIC bastion host key pair set up to do this tutorial. If you have not already set this up, follow steps 1-5 at https://github.com/fabric-testbed/teaching-materials/blob/main/Getting%20Started.md#section-1-get-started.
 

## 1. Set up the Experiment

In this section you will use the Fablib manager to create a new slice that is composed of 2 nodes for the Ryu & Mininet testbed.
### 1.1 Reserve Resources
In this section we will declare the structure of the slice we will be using.

In [ ]:
# Import Fablib
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                     
fablib.show_config()
import json
import traceback

In [ ]:
# Define and Submit Slice
slice_name = "MininetWorkspace"
site = "EDUKY"
print(site)
#The model of the VM
nicmodel = "NIC_Basic"
image = "default_ubuntu_20"

cores = 1
ram = 2
disk = 10

In [ ]:
try:
    # Create a slice
    slice = fablib.new_slice(name=slice_name)
    #first Node
    ND1 = slice.add_node(name="Host", site=site)
    ND1.set_capacities(cores=cores, ram=ram, disk=disk)
    ND1.set_image(image)
    Net_ND1 = ND1.add_component(model=nicmodel, name="ND_Host").get_interfaces()[0]
    

    #Second Node
    ND2 = slice.add_node(name="Controller", site=site)
    ND2.set_capacities(cores=cores, ram=ram, disk=disk)
    ND2.set_image(image)
    Net_ND2 = ND2.add_component(model=nicmodel, name="ND_Controller").get_interfaces()[0] 

    #Lan
    LN = slice.add_l2network(name="Lan", interfaces=[Net_ND1, Net_ND2])
    
    # Submit the slice
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")

### 1.2 Set up the Experiment Network
In this section of the slice creation, once the nodes are active, will assign the network spaces some predefined values that we will use for the assignment

In [ ]:
# Setup Network
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:
    subnet = IPv4Network("10.0.0.0/24")
    Network = 'Lan'
    
    A = slice.get_node('Host')
    B = slice.get_node('Controller')
    
    #A's lan
    A_iface = A.get_interface(network_name=Network)  
    A_iface.ip_addr_add(addr='10.0.0.1', subnet=subnet)
    #B's Lan
    B_iface = B.get_interface(network_name=Network)  
    B_iface.ip_addr_add(addr='10.0.0.2', subnet=subnet)
    
    for node in slice.get_nodes():
        for interface in node.get_interfaces():
            stdout, stderr = node.execute(f'sudo ip link set dev { interface.get_device_name()} up')
            if (node.get_name() == "ND_A"):
                A_Device_Name = interface.get_device_name()
        print(f"Network status at node {node.get_name()}")
        node.execute("ip a")
        
except Exception as e:
    print(f"Fail: {e}")

### Test the connection for the nodes
This code will ping each respective node


In [ ]:
 # Test connectivity between Host_1 and ControllerNode
print("Pinging ControllerNode from Host_1...")
A.execute('ping -c 4 10.0.0.2')  # Ping ControllerNode from Host_1
    
print("Pinging Host_1 from ControllerNode...")
B.execute('ping -c 4 10.0.0.1')  # Ping Host_1 from ControllerNode

### 1.3 Configure the software needed for the nodes in the experiment
This section of the slice creation, we will add any aditional software, tools or scripts that we need for our experiments.

In [ ]:
# Install Software including Mininet on Host
try:
    # Install common software on all nodes
    for node in slice.get_nodes():
        node.execute('yes | sudo apt-get -y update && sudo apt-get upgrade -y') 
        node.execute('sudo apt-get -y install net-tools')

    # Install Ryu controller and dependencies on the Controller node
    B.execute('sudo apt install python3-pip -y')
    B.execute('sudo apt install gcc python-dev libffi-dev libssl-dev libxml2-dev libxslt1-dev zlib1g-dev -y')
    B.execute('sudo apt-get install python3-ryu -y')
    B.execute('git clone https://github.com/faucetsdn/ryu.git')
    B.execute('git clone https://github.com/dabun01/Mininet_Ryu_Tutorial')

    # Install Mininet on Host_1
    print("Installing Mininet on Host_1...")
    A.execute('sudo apt-get -y install mininet')
    A.execute('git clone https://github.com/dabun01/Mininet_Ryu_Tutorial')
    
    print("Software installation completed successfully.")

except Exception as e:
    print(f"Fail: {e}")
    

## Continue to The experiment notebook

Once you have completed this notebook you should be able to continue to the Ryu & Mininet Notebook. You can either open it on the explorer or click [Here](./Mininet.ipynb) to open the next notebook.